# ** Xebia Part II: Machine Learning Pipeline**
0. **Pipeline**
    1. Feature union preprocessing pipeline

In [23]:
import numpy as np
import pandas as pd
import sys
import lightgbm as lgb
import xgboost as xgb

sys.path.insert(0, '../')
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn_pandas import DataFrameMapper
from src.utils import WarrantyToFloat,FillByMax

%matplotlib inline

In [2]:
df_train = pd.read_csv('../data/mower_market_snapshot.csv',sep=';',
                     na_values='unknown')
target = df_train.loc[:,"attractiveness"]
df_train = df_train.drop(["id","market_share","attractiveness"],axis=1)
df_test = pd.read_csv('../data/submission_set.csv',sep=';',na_values='unknown')

In [3]:
df_test.head(2)

,capacity,failure_rate,id,margin,price,prod_cost,product_type,quality,warranty
0,21.313064,0.037928,20049,613.061762,768.160605,155.098843,auto-portee,Medium,3 ans
1,25.797234,0.038664,19699,701.321608,865.727540,164.405932,auto-portee,Low,3 ans


In [4]:
df_train.head(2)

,capacity,failure_rate,margin,price,prod_cost,product_type,quality,warranty
0,10.492623,0.042528,633.766584,828.802103,195.035519,auto-portee,Low,3ans
1,20.273730,0.037896,731.576817,882.894916,151.318099,auto-portee,Low,3 ans.


In [5]:
p = Pipeline([
    ('preprocessing',DataFrameMapper([ 
                ('product_type', LabelBinarizer()),
                ('quality', LabelBinarizer()),
                ('warranty',WarrantyToFloat()),
                                           ],input_df=True,default=None,df_out = True)),
     ('fill_by_max',FillByMax()) # this is only used for tree methodes
    
            ])
    

In [6]:
df_train_prepro = p.fit_transform(df_train)

In [7]:
df_train_prepro.head()

,product_type_auto-portee,product_type_electrique,product_type_essence,quality_Hight,quality_Low,quality_Medium,warranty,capacity,failure_rate,margin,price,prod_cost
0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.492623,0.042528,633.766584,828.802103,195.035519
1,1.0,0.0,0.0,0.0,1.0,0.0,3.0,20.273730,0.037896,731.576817,882.894916,151.318099
2,1.0,0.0,0.0,0.0,1.0,0.0,3.0,20.190457,0.039111,741.218202,919.355522,178.137319
3,1.0,0.0,0.0,0.0,1.0,0.0,3.0,27.220131,0.041303,654.255635,817.085097,162.829462
4,1.0,0.0,0.0,0.0,0.0,1.0,3.0,16.370911,0.037938,703.094527,844.581465,141.486938


In [37]:
parameters = {'learning_rate': [0.05,0.1,0.2], #so called `eta` value
              'max_depth': [4,6,8],
              'min_child_weight': [4,8,12],
              'subsample': [0.6,0.7,0.8],
              'colsample_bytree': [0.6,0.7,0.8],
              'n_estimators': [30,50,80], #number of trees, change it to 1000 for better results
               }

xgb_model = xgb.XGBRegressor(objective='reg:linear')
clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=5, 
                   scoring='neg_mean_squared_error',
                   verbose=1, refit=True,return_train_score=True)

_ = clf.fit(df_train_prepro,target)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:    2.9s
[Parallel(n_jobs=5)]: Done 1090 tasks      | elapsed:   21.6s
[Parallel(n_jobs=5)]: Done 2590 tasks      | elapsed:   53.6s
[Parallel(n_jobs=5)]: Done 3645 out of 3645 | elapsed:  1.3min finished


In [36]:
clf.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 4,
 'n_estimators': 50,
 'subsample': 0.7}

In [35]:
clf.best_score_

-0.012095698950621554